In [ ]:
%run vis/xAPISG-noDataToFillVisualization.ipynb # notebook to create the visualisation with a message NoDataToFill
%run vis/helpersFunctions/barCharts.ipynb
%run vis/helpersFunctions/OrderGraphValuesAndSelectDataByXaxis.ipynb

***vis_alternatives_selected_question* function**:
* Output : the widgets selector and the outputAlternativeQuestion in a list to be display in a VBox
* outputAlternativeQuestion : display a bar chart showing the alternatives selcted in each multiple-choice question

In [ ]:
def vis_alternatives_selected_question():
    #constructing a dropdown to order for alphabetic keys or for values
    orderAlternative=widgets.Dropdown(options=[('alphabetic',0),('value from highest to lowest',1),('value from lowest to highest',-1)],
                                description='Order',
                                disabled=False)
    def display_alternatives_selected_question(change):
        with outputAlternativeQuestion:
            clear_output(wait=False)
            alternatives_selected_question(orderAlternative.value)
            clear_output(wait=True)
    orderAlternative.observe(display_alternatives_selected_question, 'value')
    display_alternatives_selected_question(None)
    return [orderAlternative, outputAlternativeQuestionUpdate, outputAlternativeQuestion]

***alternatives_selected_question* function**:

Input:
   * order : dropdown value indicating how to order xlabel : alphabetically or by y value (hight to low or low to hight)

Output : Displays bar chart showing the alternatives selcted in each multiple-choice question

In [ ]:
def alternatives_selected_question(order):
    #close and clear previous fig of altenatives
    for fig in figResPerAlt:
        plt.close(fig=fig)
    figResPerAlt.clear()
    playersSelected=checkboxesPlayersSelected.get_selected_options()
    alternatives_list_selected=checkboxesAlternativeSelected.get_selected_options()
    if len(alternatives_list_selected)>0 and len(playersSelected)>0:
        accordion_alternatives=widgets.Accordion(children=[widgets.Output() for key in alternatives_list_selected],
                                                 selected_index=None)
        i=0
        for alternative in alternatives_list_selected:
            accordion_alternatives.set_title(i, "Responses selected to question "+ alternative)
            i+=1
        display(accordion_alternatives)
        def on_selected_index_alternative_change(change):
            new=change['new']
            if new!=None:
                #close and clear previous fig of altenatives
                for fig in figResPerAlt:
                    plt.close(fig=fig)
                figResPerAlt.clear()
                #get all players and alternatives selected
                playersSelected=checkboxesPlayersSelected.get_selected_options()
                alternatives_list_selected=checkboxesAlternativeSelected.get_selected_options()
                alt=alternatives_list_selected[new]
                # responses per alternative
                selected_per_alternative = {}
                total_max=[]
                i=0
                selected = []
                total_max.insert(i, 0)
                for player in playersSelected:
                    if alt in players_info[player]["alternatives"].keys():
                        alternative, _ = zip(*players_info[player]["alternatives"][alt])
                        alternative=list(alternative)
                        selected.extend(alternative)
                selected_per_alternative[alt] = Counter(selected)
                for res in selected_per_alternative[alt].values():
                    if res>total_max[i]:
                        total_max[i]=res
                total_max[i]+=0.5
                i+=1
                #Good answer per alternative
                goodAnswer=[]
                for player in players_info.keys():
                    if alt in players_info[player]["alternatives"].keys():
                        for (answer, success) in players_info[player]["alternatives"][alt]:
                            if success in ['True', True]:
                                goodAnswer.append(answer)
                #order alls values
                selected_per_alternative = orderDictByDictValues(selected_per_alternative,order)
                with accordion_alternatives.children[new]:
                    clear_output(wait=False)
                    #create new fig with custom name in the tab
                    name=filename+"_selected_answers_per_questions_"+str(alt)
                    fig = plt.figure(num=name, constrained_layout=True)
                    fig.canvas.header_visible = False
                    fig.canvas.layout.min_height = '400px'
                    figResPerAlt.append(fig)
                    #display bar chart
                    barlist=plt.bar(range(len(selected_per_alternative[alt])),
                           selected_per_alternative[alt].values(),
                           tick_label=list(selected_per_alternative[alt].keys()))
                    add_value_labels(barlist)
                    plt.title('Responses selected to question '+alt)
                    j=0
                    for answer in selected_per_alternative[alt].keys():
                        if answer in goodAnswer:
                            barlist[j].set_color('g')
                        else:
                            barlist[j].set_color('r')
                        j+=1
                    plt.yticks(np.arange(0, total_max[0], 1))
                    plt.ylim(0,total_max[0])
                    #add axes labels and title
                    plt.xlabel("Responses selected")
                    plt.ylabel("Number of responses")
                    #orient xticks
                    plt.xticks(rotation=90)
                    plt.show()
        accordion_alternatives.observe(on_selected_index_alternative_change, 'selected_index')
    else:
        #create new fig with custom name in the tab
        name=filename+"_selected_answers_per_questions"
        fig = plt.figure(num=name, constrained_layout=True)
        figResPerAlt.append(fig)
        #display no data to fill vis
        noDataToFillVis(100)
        #add axes labels and title
        plt.title("Responses selected to question")
        plt.xlabel("Responses selected")
        plt.ylabel("Number of responses")
        plt.show()
    with outputAlternativeQuestionUpdate:
        clear_output(wait=False)